In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import pandas as pd

/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
INDEX_NAME = "bills"
INDEX_FILE = "index.json"

DATA_FILE = "nc-legislation-data-2019-2020-session.xlsx"

In [3]:
def connect_elasticsearch():
    client = None
    client = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if client.ping():
        print('Connected')
    else:
        print('Connection Error')
    return client

In [4]:
client = connect_elasticsearch()

Connected


In [5]:
client.indices.delete(index=INDEX_NAME, ignore=[404])

{'acknowledged': True}

In [6]:
with open(INDEX_FILE) as index_file:
    source = index_file.read().strip()
    client.indices.create(index=INDEX_NAME, body=source)

In [7]:
data = pd.read_excel(DATA_FILE)

In [8]:
data

,bill_id,alternate_bill_id,bill_name,bill_text_url,bill_summary,bill_sponsors,action_on_branch,action_date,action_text,body_text
0,H1,NaN,2019 House Temporary Rules.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,David R. Lewis,H,2019-01-09,Adopted,SECTION 1. The temporary rules of the Regular...
1,H10,S19,Repeal 2015 Law Relating to Monuments.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,Pricey Harrison; Evelyn Terry; Marcia Morey; K...,H,2019-05-21,"Re-ref Com On Rules, Calendar, and Operations ...",SECTION 1. Subsections (a) through (e) of Sec...
2,H100,NaN,Required Training Police Telecommunicators.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,John Faircloth; John A. Torbett; Carson Smith;...,S,2019-09-30,Re-ref Com On Appropriations/Base Budget,SECTION 1. G.S.17E7 reads as rewritten: (a) J...
3,H1000,NaN,Competency-Based Math Pilot.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,Pat B. Hurley; Jason Saine; Hugh Blackwell,H,2019-04-29,"Ref to the Com on Education - K-12, if favorab...",SECTION 1. Purpose. There is established the...
4,H1001,S407,Raise the Age Funding.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,"James L. Boles, Jr.; Gregory F. Murphy, MD; Ca...",H,2019-10-01,Placed On Cal For 10/02/2019,SECTION 1. There is appropriated from the Gen...
5,H1002,NaN,Expand Use of CAM Systems.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,David Rogers,S,2019-08-16,"Re-ref to Transportation. If fav, re-ref to Ju...",SECTION 1. G.S.20179 reads as rewritten: (g)...
6,H1003,NaN,Funds for Growing High Point.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,Cecil Brockman; John Faircloth; Jon Hardister;...,H,2019-04-29,"Ref to the Com on Appropriations, General Gove...",SECTION 1. There is appropriated from the Gen...
7,H1004,NaN,High Point LEAP/Funds for Literacy Programs.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,Cecil Brockman; Ashton Wheeler Clemmons; John ...,H,2019-04-29,"Ref to the Com on Appropriations, Education, i...",SECTION 1. There is appropriated from the Gen...
8,H1005,NaN,Funds to Help High Point Reduce Violence.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,Cecil Brockman; Pricey Harrison; John Fairclot...,H,2019-04-29,"Ref to the Com on Appropriations, General Gove...",SECTION 1. There is appropriated from the Gen...
9,H1006,NaN,High Point Economic Development.,https://www.ncleg.gov/sessions/2019/bills/hous...,https://lrs.sog.unc.edu/bill-summaries-lookup/...,"Cecil Brockman; Amos L. Quick, III; John Fairc...",H,2019-04-29,"Ref to the Com on Appropriations, Agriculture ...",SECTION 1. There is appropriated from the Gen...


In [9]:
!ls

NC_Bill_Text
Untitled.ipynb
index.json
nc-legislation-data-2019-2020-session.xlsx
~$nc-legislation-data-2019-2020-session.xlsx


In [10]:
import shlex
import subprocess

In [11]:
requests = []
for i, row in data.iterrows():
    request = row
    #request["_op_type"] = "index"
    #request["_index"] = INDEX_NAME
    #requests.append(request.to_json())
#     print(request["body_text"])
#     break
    cmd = '''
    curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}'
    '''
    args = shlex.split(cmd)
    process = subprocess.Popen(args, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    client.index(index=INDEX_NAME,doc_type="_doc",id=i,body=request.to_json())
#bulk(client, requests)

In [16]:
search = "education"
query = {
    "query":{
       "more_like_this":{
          "fields":[
             "body_text"
          ],
          "like":search,
          "min_term_freq":1,
          "max_query_terms":20
       }
    }
}

In [46]:
query = {'query': {'match': {'bill_id': 'H1'}}}

In [17]:
client.search(index=INDEX_NAME, body=query)

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 420, 'relation': 'eq'},
  'max_score': 2.948927,
  'hits': [{'_index': 'bills',
    '_type': '_doc',
    '_id': '1422',
    '_score': 2.948927,
    '_source': {'bill_id': 'S460',
     'alternate_bill_id': None,
     'bill_name': 'Performance/Professors as K-12 Teachers.',
     'bill_text_url': 'https://www.ncleg.gov/sessions/2019/bills/senate/html/S460v1.html',
     'bill_summary': 'https://lrs.sog.unc.edu/bill-summaries-lookup/S/460/2019-2020%20Session/S460',
     'bill_sponsors': 'Don Davis;\xa0Deanna Ballard;\xa0Erica D. Smith',
     'action_on_branch': 'S',
     'action_date': 1554163200000,
     'action_text': 'Ref To Com On Rules and Operations of the Senate',
     'body_text': ' SECTION 1. G.S.115C157.1 reads as rewritten: (a) Adjunct Hiring Criteria.   The State Board of Education shall develop minimum criteria of relevant  education or employment 

In [ ]:
json

In [29]:
requests

[bill_id                                                             H1
 alternate_bill_id                                                  NaN
 bill_name                                  2019 House Temporary Rules.
 bill_text_url        https://www.ncleg.gov/sessions/2019/bills/hous...
 bill_summary         https://lrs.sog.unc.edu/bill-summaries-lookup/...
 bill_sponsors                                           David R. Lewis
 action_on_branch                                                     H
 action_date                                        2019-01-09 00:00:00
 action_text                                                    Adopted
 _op_type                                                         index
 _index                                                           bills
 Name: 0, dtype: object,
 bill_id                                                            H10
 alternate_bill_id                                                  S19
 bill_name                       Repeal